## Examples of usage

In [35]:
import numpy as np
import random
import pandas as pd

In [ ]:
from meliora import jeffreys_test

# Create a random dataset
ratings = random.choices(['A', 'B', 'C'],  [0.4, 0.5, 0.1], k=1000)
default_flag = random.choices([0, 1],  [0.9, 0.1], k=1000)
probs_default = [np.clip(random.normalvariate(0.1, 0.05), 0, 1) for x in range(1000)]
test_data = pd.DataFrame({'ratings': ratings, 
                                'default_flag': default_flag, 
                                'predicted_pd' : probs_default})

# Test Jeffreys test
from meliora import jeffreys_test
jeffreys_test(test_data, 'ratings', 'default_flag', 'probs_default')

# Test Jeffrey's test
df = pd.read_csv('pd_test_data.csv')
result= jeffreys_test(df, 'ratings', 'default_flag', 'predicted_pd', 0.05)
result

In [ ]:
# Jeffrey's test
jeffreys_test(test_data, "ratings", "default_flag", "predicted_pd", alpha_level=0.1)

In [ ]:
# Binomial test
binomial_test(test_data, "ratings", "default_flag", "predicted_pd")

In [ ]:
# Brier score
brier_score(test_data, "ratings", "default_flag", "predicted_pd")

In [ ]:
# Herfindahl test

ratings2 = random.choices(buckets,  [0.4, 0.5, 0.1], k=1000)
probs_default2 = [bucket_pds[rating] for rating in ratings2]
default_flag2 = [random.uniform(0, 1) < bucket_pds[rating]
                for rating in ratings2]
test_data2 = pd.DataFrame({'ratings': ratings2, 
                         'default_flag': [int(i) for i in default_flag2], 
                         'predicted_pd' : probs_default2})

test_data2.to_csv('pd_test_data2.csv', index=False)

herfindahl_test(test_data, test_data2, "ratings")

In [ ]:
# Hosmer-Lemeshow test
hosmer_test(test_data, "ratings", "default_flag", "predicted_pd")

In [ ]:
# Spiegelhalter test
spiegelhalter_test(test_data, "ratings", "default_flag", "predicted_pd")